In [11]:
import keras
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.regularizers import l2, activity_l2,l1
from keras.utils import np_utils
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.applications.vgg16 import preprocess_input
from scipy import misc
from os import listdir
import numpy as np
import matplotlib.pyplot as plt
import scipy

### Generate model with VGG16 feature extractors, set trainable false

In [56]:
def getModel( output_dim ):
    # output_dim: the number of classes (int)
    # return: compiled model (keras.engine.training.Model)
    
    vgg_model = VGG16( weights='imagenet', include_top=True )
    vgg_out = vgg_model.layers[-2].output 
    

    softmax = Dense( output_dim, activation="softmax", W_regularizer = l2(0.02))( vgg_out )

    tl_model = Model( input=vgg_model.input, output=softmax )
    # Transfer Learning
    for layer in tl_model.layers[0:-1]:
        layer.trainable = False            

    tl_model.compile(loss= "categorical_crossentropy", optimizer="adadelta", 
                     metrics=["acc"])
    
    return tl_model

### Loading all images into 'album'

In [40]:
def loadImages(path = '/mnt/cube/UT_/Urban_tribes/'):
    album = {}
    for item in listdir(path):
        category = item[0:4]
        if category == ".ipy":
            continue
        if category not in album:
            album[category] = []
        
        img = load_img(path+item)
        img = img_to_array(img)
        img = misc.imresize(img, (224, 224))
        img = scipy.misc.imrotate(img, 180)
        album[category].append(img)
    return album

In [4]:
plt.imshow(album['goth'][0][:, :, 0])
plt.show()

NameError: name 'album' is not defined

### Split 'album' into dataset
Album is split into training and testing input/outputs according to number of examples attribute

In [41]:
def make_dataset(album, numExamples):
    trn_inp = []
    trn_out = []
    tst_inp = []
    tst_out = []
    keys = album.keys()
    for key in keys:
        examples = album[key]
        l = len(examples)
        idx = np.random.choice(l, numExamples+5)
        for i in idx[:-5]:
            trn_inp.append(examples[i])
            trn_out.append(keys.index(key))
        for i in idx[-5:]:
            tst_inp.append(examples[i])
            tst_out.append(keys.index(key))
    return [trn_inp, trn_out, tst_inp, tst_out]      

In [42]:
album = loadImages()

In [43]:
plt.imshow(album['bike'][0][:, :, 0])
# plt.show()
album.keys()

['surf',
 'goth',
 'hiph',
 'form',
 'club',
 'heav',
 'othe',
 'bike',
 'rave',
 'hips',
 'coun']

In [44]:
dataset = make_dataset(album, 16)

In [45]:
trainX = np.array(dataset[0])
trainY = np.array(dataset[1])

# Now shuffle the training data and swapaxes
idx = np.random.choice(len(trainX), len(trainX))
trainX = trainX[idx]
trainY = trainY[idx]

trainX = preprocess_input(np.float64(trainX)).swapaxes(1, 3).swapaxes(2, 3)
trainY = np_utils.to_categorical(trainY)

# Only swapaxes for testing data
testX = np.array(dataset[2])
testY = np.array(dataset[3])

testX = preprocess_input(np.float64(testX)).swapaxes(1, 3).swapaxes(2, 3)
testY = np_utils.to_categorical(testY)

In [47]:
plt.imshow(trainX[10][0, :, :])
plt.show()

### Fit the model

In [ ]:
model = getModel(11)

In [ ]:
history = model.fit(trainX, trainY, batch_size = 16, nb_epoch = 30, validation_data = (testX, testY), shuffle=True)

Train on 176 samples, validate on 55 samples
Epoch 1/30
176/176 [==============================] - 11s - loss: 3.0697 - acc: 0.2273 - val_loss: 3.1058 - val_acc: 0.1818
Epoch 2/30
176/176 [==============================] - 10s - loss: 1.7716 - acc: 0.5966 - val_loss: 2.8761 - val_acc: 0.1455
Epoch 3/30
176/176 [==============================] - 11s - loss: 1.2621 - acc: 0.8011 - val_loss: 2.7514 - val_acc: 0.2364
Epoch 4/30
176/176 [==============================] - 10s - loss: 0.9856 - acc: 0.8977 - val_loss: 2.7162 - val_acc: 0.2727
Epoch 5/30
176/176 [==============================] - 11s - loss: 0.8262 - acc: 0.9148 - val_loss: 2.6335 - val_acc: 0.2545
Epoch 6/30
176/176 [==============================] - 10s - loss: 0.7228 - acc: 0.9545 - val_loss: 2.5927 - val_acc: 0.2727
Epoch 7/30
 96/176 [===============>..............] - ETA: 3s - loss: 0.6661 - acc: 0.9583